<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Asset Management

### MVP Portfolio Analysis Class

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [training@tpq.io](mailto:trainin@tpq.io) | [@dyjh](http://twitter.com/dyjh)

### The use of the "Python 3.10, Numpy 1.26.4" kernel is recommended.

## Standardizing Tasks

Topics of interest include:

* Importing, visualizing the data
* `FinancialData` base class
* `MVPPortfolio` portfolio class

## Real Data

**_Historical end-of-day financial time series data._**

Data retrieved vom Refinitiv Eikon for the German DAX 30 constituents.

The data sets:

    http://hilpisch.com/dax_eikon_eod_data.csv
    http://hilpisch.com/dax_eikon_mc_data.csv

## Imports and Data

In [ ]:
import math
import cufflinks
import eikon as ek
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn-v0_8')
cufflinks.set_config_file(offline=True)
np.set_printoptions(suppress=True, precision=4)
pd.options.display.float_format = '{:.5f}'.format
%config InlineBackend.figure_format = 'svg'

In [ ]:
url = 'http://hilpisch.com/dax_eikon_eod_data.csv'

In [ ]:
raw = pd.read_csv(url, index_col=0)

In [ ]:
raw.iloc[:5, :5]

In [ ]:
universe = raw.columns# [-10:]
universe

## Financial Data Class

Methods to be implemented:

    .__init__()
    .retrieve_data()
    .prepare_data()
    .plot_data()
    .plot_mc()
    .plot_corr()

In [ ]:
class FinancialData:
    url = 'http://hilpisch.com/dax_eikon_eod_data.csv'
    url_ = 'http://hilpisch.com/dax_eikon_mc_data.csv'
    def __init__(self, universe):
        self.universe = universe
        self.no_assets = len(universe)
        self.retrieve_data()
        self.prepare_data()
    def retrieve_data(self):
        self.raw = pd.read_csv(self.url, index_col=0)
        self.raw_ = pd.read_csv(self.url_, index_col=0)
    def prepare_data(self):
        self.data = self.raw[self.universe]
        self.rets = np.log(self.data / self.data.shift(1))
        self.mc = (self.raw_.T[self.universe]).T
        self.mc['MC%'] = self.mc['MC'].apply(lambda x: x / self.mc['MC'].sum())
    def plot_data(self, cols=None):
        if cols is None:
            cols = self.universe
        self.data[cols].normalize().iplot()
    def plot_mc(self):
        self.mc.sort_values('MC').iplot(kind='pie',
                values='MC', labels='NAME', colorscale='rdylbu')
    def plot_corr(self):
        self.rets.corr().iplot(kind='heatmap', colorscale='reds')

In [ ]:
fd = FinancialData(universe)

In [ ]:
# fd.raw

In [ ]:
# fd.raw_.T

In [ ]:
# fd.mc

In [ ]:
fd.plot_data()

In [ ]:
fd.plot_mc()

In [ ]:
fd.plot_corr()

## MVP Class

Methods to be implemented:

    .__init__()
    .portfolio_return()
    .portfolio_variance()
    .portfolio_volatility()
    .portfolio_sharpe()
    ._set_bounds_constraints()
    ._get_results()
    .minimum_volatility_portfolio()
    .maximum_sharpe_portfolio()
    .plot_weights()
    .plot_performance()

In [ ]:
universe = raw.columns[-15:]
universe

In [ ]:
from scipy.optimize import minimize

In [ ]:
class MVPPortfolio(FinancialData):
    def __init__(self, universe):
        super().__init__(universe)
        self.equal_weights = self.no_assets * [1 / self.no_assets]
        self.mc_weights = self.mc['MC%'].values
    def portfolio_return(self, weights, days=252):
        return np.dot(self.rets.mean(), weights) * days
    def portfolio_variance(self, weights, days=252):
        return np.dot(weights, np.dot(self.rets.cov(), weights)) * days
    def portfolio_volatility(self, weights, days=252):
        return math.sqrt(self.portfolio_variance(weights, days))
    def portfolio_sharpe(self, weights, days=252):
        sharpe = (self.portfolio_return(weights, days) /
                  self.portfolio_volatility(weights, days))
        return sharpe
    def _set_bounds_constraints(self, bnds, cons):
        if bnds is None:
            self.bnds = self.no_assets * [(0, 1)]
        else:
            self.bnds = bnds
        if cons is None:
            self.cons = {'type': 'eq', 'fun': lambda weights: weights.sum() - 1}
        else:
            self.cons = cons
    def _get_results(self, opt, kind):
        ret = self.portfolio_return(opt['x'])
        vol = self.portfolio_volatility(opt['x'])
        sharpe = self.portfolio_sharpe(opt['x'])
        weights = pd.DataFrame(opt['x'], index=self.universe, columns=['weights',])
        res = {'kind': kind, 'weights': weights.round(7), 'return': ret,
               'volatility': vol, 'sharpe': sharpe}
        return res
    def minimum_volatility_portfolio(self, bnds=None, cons=None):
        self._set_bounds_constraints(bnds, cons)
        opt = minimize(self.portfolio_volatility, self.equal_weights,
                      bounds=self.bnds, constraints=self.cons)
        self.results = self._get_results(opt, 'Minimum Volatility')
        return self.results
    def maximum_sharpe_portfolio(self, bnds=None, cons=None):
        self._set_bounds_constraints(bnds, cons)
        tf = lambda weights: -self.portfolio_sharpe(weights)
        opt = minimize(tf, self.equal_weights, bounds=self.bnds,
                       constraints=self.cons)
        self.results = self._get_results(opt, 'Maximum Sharpe')
        return self.results
    def plot_weights(self, kind='pie'):
        if kind == 'pie':
            nonzero = self.results['weights'] > 0
            to_plot = self.results['weights'][nonzero['weights']].copy()
            to_plot['names'] = to_plot.index
            to_plot.iplot(kind='pie', values='weights',
                          labels='names', colorscale='rdylbu',
                          title='Optimal Weights | ' + self.results['kind'])
        else:
            self.results['weights'].iplot(kind='bar',
                    title='Optimal Weights | ' + self.results['kind'])
    def plot_performance(self):
        perf = (self.results['return'], self.results['volatility'], self.results['sharpe'])
        index = ['return', 'volatility', 'sharpe']
        to_plot = pd.DataFrame(perf, index=index, columns=['metrics',])
        to_plot.iplot(kind='bar', title='Performance Metrics  | ' + self.results['kind'])

In [ ]:
mvp = MVPPortfolio(universe)

In [ ]:
# mvp.equal_weights

In [ ]:
mvp.portfolio_return(mvp.equal_weights)

In [ ]:
mvp.portfolio_volatility(mvp.equal_weights)

In [ ]:
mvp.portfolio_sharpe(mvp.equal_weights)

In [ ]:
mvp.mc_weights

In [ ]:
mvp.portfolio_return(mvp.mc_weights)

In [ ]:
mvp.portfolio_volatility(mvp.mc_weights)

In [ ]:
mvp.portfolio_sharpe(mvp.mc_weights)

In [ ]:
res = mvp.minimum_volatility_portfolio()

In [ ]:
res['weights']

In [ ]:
res['return']  # expected return

In [ ]:
res = mvp.maximum_sharpe_portfolio(bnds=None)

In [ ]:
res['weights']

### Minimum Volatility

In [ ]:
res = mvp.minimum_volatility_portfolio()

In [ ]:
res['weights']

In [ ]:
mvp.plot_weights()

In [ ]:
mvp.plot_performance()

### Maximum Sharpe Ratio 

In [ ]:
res = mvp.maximum_sharpe_portfolio()

In [ ]:
res['weights']

In [ ]:
mvp.plot_weights()

In [ ]:
mvp.plot_weights(kind='bar')

### Bounded Maximum Sharpe

In [ ]:
bnds = mvp.no_assets * [(0.01, 0.4),]

In [ ]:
res = mvp.maximum_sharpe_portfolio(bnds=bnds)

In [ ]:
res['weights']

In [ ]:
mvp.plot_weights()

In [ ]:
mvp.plot_weights(kind='bar')

### Short Sales Allowed

In [ ]:
bnds = mvp.no_assets * [(-1, 1),]

In [ ]:
cons = {}

In [ ]:
res = mvp.maximum_sharpe_portfolio(bnds=bnds, cons=cons)

In [ ]:
res['weights']

In [ ]:
mvp.plot_weights(kind='bar')

In [ ]:
mvp.plot_performance()

## Exercise

**Use the `MVPPortfolio` class and adjust it in a way that it becomes a `BL92Portfolio` class.**

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>